<a href="https://colab.research.google.com/github/Dowell-Lab/psea/blob/main/notebook_examples/simulateddata-bothdirs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import numpy.random as random
from datetime import datetime
import random
import plotly.express as px
from google.colab import drive


In [2]:
now = datetime.now()
date_time_string = now.strftime("%Y%m%d%H%M%S")
print(date_time_string)
outdir="/content/drive/MyDrive/temp/"

20241010201659


In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Look at the real data

In [4]:
#This brings in the normalized counts for all the individuals with Trisomy 21 in the Human Trisome Project. These are not the real count data but are similar to reall count data.
#df=pd.read_csv('/content/drive/MyDrive/normcounts.csv')
gene_exp_url="https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/value_expression.csv"
gene_exp_df=pd.read_csv(gene_exp_url, index_col=0)

#this brings in the medical disorders for all the individuals with Trisomy 21 in the Human Trisome Project
comorbid_url = "https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/comorbid_file.csv"
comorbid_df = pd.read_csv(comorbid_url, index_col=0)

#this brings in random names to use
namefile = "https://raw.githubusercontent.com/Dowell-Lab/psea/refs/heads/main/testdata/namelist.txt"
namesdf = pd.read_csv(namefile, index_col=0, names=["name"])



In [5]:
samplename="Patient" # What is the sample column name in your data. In mine, it is patient.

## How many samples does the real data have

In [6]:
total_samples = gene_exp_df.shape[0]


## Plot the real gene expression data

Collect the metadata about the values and binaray attributes. Remove values where the mean is 0. Remove bianarys where the samples with the attribute are all samples or none.

In [7]:
def collect_gene_metadata(gene_exp_df):
  genenames = [colname for colname in gene_exp_df.columns if colname!=samplename]
  gene_exp_df_metadata = gene_exp_df[genenames]
  gene_exp_df_metadata = gene_exp_df_metadata.T
  gene_exp_df_metadata["mean"] = gene_exp_df_metadata.mean(axis=1)
  gene_exp_df_metadata["std"] = gene_exp_df_metadata.std(axis=1)
  gene_exp_df_metadata = gene_exp_df_metadata[["mean", "std"]]
  gene_exp_df_metadata = gene_exp_df_metadata[gene_exp_df_metadata["mean"]!=0]
  gene_exp_df_metadata["log_mean"] = np.log(gene_exp_df_metadata["mean"])
  gene_exp_df_metadata["log_std"] = np.log(gene_exp_df_metadata["std"])
  return gene_exp_df_metadata


def collect_comorbid_metadata(comorbid_df):
  comorbidnames = [colname for colname in comorbid_df.columns if colname!=samplename]
  comorbid_df_metadata = comorbid_df[comorbidnames].T
  comorbid_df_metadata["samples_with_binary_attribute"] = comorbid_df_metadata.sum(axis=1)
  comorbid_df_metadata = comorbid_df_metadata[["samples_with_binary_attribute"]]
  return comorbid_df_metadata

def removeallornone_bianarys(comorbid_df_metadata):
  comorbid_df_metadata = comorbid_df_metadata[comorbid_df_metadata["samples_with_binary_attribute"]!=total_samples]
  comorbid_df_metadata = comorbid_df_metadata[comorbid_df_metadata["samples_with_binary_attribute"]!=0]
  return comorbid_df_metadata


In [9]:
gene_exp_df_metadata = collect_gene_metadata(gene_exp_df)

In [10]:
fig = px.scatter(gene_exp_df_metadata, y="log_mean", x="log_std")
fig.show()


## Plot the read comorbidity data

In [11]:
comorbid_df_metadata = collect_comorbid_metadata(comorbid_df)

In [12]:
fig = px.violin(comorbid_df_metadata, y="samples_with_binary_attribute")
fig.show()

In [13]:
comorbid_df_metadata = removeallornone_bianarys(comorbid_df_metadata)
fig = px.violin(comorbid_df_metadata, y="samples_with_binary_attribute", box=True)
fig.show()

# Code that makes the simulated data

## Pick names for simulated samplesnames

In [14]:
# Select a random sample of names from namesdf
random_names = namesdf.sample(n=total_samples, replace=False)
random_names["name"] = random_names.index


### How many genes do you want to simulate

In [15]:
simulate_n_genes = 10

## First simulate the simulated values on the real values in the valuedf (gene expression based on real genes)

In [16]:
def variableexp(total_samples, mean_exp, std_exp):
  arr = np.random.normal(mean_exp, std_exp, total_samples)
  return arr

def generate_gene_exp(simulated_gene_exp_df):
  simulated_gene_exp_df["exp_array"] =  simulated_gene_exp_df.apply(lambda row: variableexp(total_samples, row["mean"], row["std"]), axis=1)
  return simulated_gene_exp_df

def simulate_values_based_on_real_genes(gene_exp_df_metadata, simulate_n_genes, total_samples):
  # Cut the data from the 'mean' column into simulate_n_genes bins with equal number of rows
  gene_exp_df_metadata['exp_group'] = pd.qcut(gene_exp_df_metadata['mean'], q=simulate_n_genes, labels=False)

  # Create an empty DataFrame to store the randomly selected rows
  simulated_gene_exp_df = pd.DataFrame()

  # Iterate through each unique group in 'exp_group'
  for group in gene_exp_df_metadata['exp_group'].unique():
    # Select rows belonging to the current group
    rows_in_group = gene_exp_df_metadata[gene_exp_df_metadata['exp_group'] == group]
    # Randomly select one row from the group
    if not rows_in_group.empty:
      random_row = rows_in_group.sample(n=1)
      # Append the randomly selected row to the DataFrame
      simulated_gene_exp_df = pd.concat([simulated_gene_exp_df, random_row])
  simulated_gene_exp_df = generate_gene_exp(simulated_gene_exp_df)
  simulated_gene_exp_df["sim_gene_name"] = "simulated_based_on_"+simulated_gene_exp_df.index
  repeat_random_names = [random_names["name"].to_list() for i in range(simulate_n_genes)]
  simulated_gene_exp_df["names"] = repeat_random_names
  simulated_gene_exp_df = simulated_gene_exp_df[["names", "exp_array", "sim_gene_name"]].copy()
  simulated_gene_exp_df_long = simulated_gene_exp_df.explode(["names", "exp_array"])
  final_simulated_value_df = simulated_gene_exp_df_long.pivot(index='names', columns='sim_gene_name', values='exp_array')
  final_simulated_value_df.index.name = None
  return final_simulated_value_df



In [17]:
simulated_value_df = simulate_values_based_on_real_genes(gene_exp_df_metadata, simulate_n_genes, total_samples)

In [18]:
simulated_value_df

sim_gene_name,simulated_based_on_ENSG00000182362,simulated_based_on_ENSG00000186866,simulated_based_on_ENSG00000205929,simulated_based_on_ENSG00000215533,simulated_based_on_ENSG00000227702,simulated_based_on_ENSG00000265590,simulated_based_on_ENSG00000274060,simulated_based_on_ENSG00000275631,simulated_based_on_ENSG00000276612,simulated_based_on_ENSG00000279313
Adelaide,762.284473,1761.359559,2.015837,-2.355501,-0.063328,-5.238028,1.305539,0.279989,2.925324,0.40501
Alene,274.758457,2119.074844,3.383133,53.136979,0.336747,-11.035723,0.996769,0.087215,1.377229,1.059928
Alethea,780.420736,1945.972034,-0.75995,104.019999,-0.00619,8.671733,0.941949,-0.131428,-0.907956,-0.070468
Alex,388.88171,1679.400315,-0.775175,-3.01524,0.129642,4.213551,1.767994,-0.1178,7.971818,-0.67325
Alfreda,422.306943,1783.627809,0.506168,146.348992,-0.047868,21.934044,1.150097,0.160869,-8.413809,0.800292
...,...,...,...,...,...,...,...,...,...,...
Viviana,576.896017,2335.103047,1.960536,77.699073,-0.362702,26.304445,1.747351,0.019832,2.444285,-0.013177
Willette,516.38912,1773.502377,2.007019,0.921093,0.017689,14.022951,2.76315,0.016522,1.627246,0.021459
Wynne,632.786114,2186.381287,0.27576,-16.948605,-0.124523,16.387029,2.03171,-0.239765,4.376065,-0.082825
Yettie,420.387074,1963.235011,1.694984,77.729248,-0.128071,10.985501,2.290888,0.081103,13.06964,-0.051963


In [19]:
simulated_value_df.to_csv(outdir+"simulated_gene_exp_"+date_time_string+".csv")

## Simulate the comorbids

In [26]:
def make_binary_list(total_samples, samples_true_uniform):
  binary_list = [0] * total_samples
  for _ in range(samples_true_uniform):
    random_index = random.randint(0, total_samples - 1)
    while binary_list[random_index] == 1:
      random_index = random.randint(0, total_samples - 1)
    binary_list[random_index] = 1
  return binary_list

def find_nearest_index(array, value):
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return idx


def add_bias_one_set_values(current_binary_list, simulated_value_df, genename, percent_binary_attributes_thatarevaluebias, Zscore_valuebais, Zscore_valuebais_sigma, top_or_bottom, samples_true):
  gene_vals = sorted(simulated_value_df[genename].to_list())
  gene_mean = simulated_value_df[genename].mean()
  gene_std = simulated_value_df[genename].std()
  gene_target_sigma = Zscore_valuebais_sigma*gene_std
  if top_or_bottom=="top":
    gene_target = gene_mean+Zscore_valuebais*gene_std
  else:
    gene_target = gene_mean-Zscore_valuebais*gene_std
  while sum(current_binary_list)<samples_true:
    random_number = np.random.normal(loc=gene_target, scale=gene_target_sigma)
    index_nearest_to_random_number = find_nearest_index(gene_vals, random_number)
    if current_binary_list[index_nearest_to_random_number]!=1:
      current_binary_list[index_nearest_to_random_number] = 1
  return current_binary_list


def create_simulated_n_biarary_att_from_realdata(comorbid_df_metadata, nlevels_binaryatts=12):
  nbinaryatt_min = comorbid_df_metadata["samples_with_binary_attribute"].min()
  nbinaryatt_max =comorbid_df_metadata["samples_with_binary_attribute"].max()
  nbinaryatt_med = comorbid_df_metadata["samples_with_binary_attribute"].median()
  nbinaryatt_std = comorbid_df_metadata["samples_with_binary_attribute"].std()
  simulated_n_bianary_attributes = []
  simulated_n_bianary_attributes.append(nbinaryatt_min)
  simulated_n_bianary_attributes.append(nbinaryatt_max)
  while len(simulated_n_bianary_attributes)<nlevels_binaryatts:
    random_number = int(np.random.normal(loc=nbinaryatt_med, scale=nbinaryatt_std))
    if random_number>0:
      simulated_n_bianary_attributes.append(random_number)
      simulated_n_bianary_attributes = sorted(list(set(simulated_n_bianary_attributes)))
  return simulated_n_bianary_attributes


def create_many_ba(simulated_value_df, percent_binary_attributes_thatarevaluebias_list, Zscore_valuebais_list, Zscore_valuebais_sigma_list, simulated_n_bianary_attributes, biasdirs):
  df_list =[]
  for percent_binary_attributes_thatarevaluebias in percent_binary_attributes_thatarevaluebias_list:
    for Zscore_valuebais in Zscore_valuebais_list:
      for Zscore_valuebais_sigma in Zscore_valuebais_sigma_list:
        for samples_true in simulated_n_bianary_attributes:
          for top_or_bottom in biasdirs:
            for genename in simulated_value_df.columns:
              percent_binary_attributes_not_bias = 1-percent_binary_attributes_thatarevaluebias
              samples_true_uniform = int(round(samples_true*percent_binary_attributes_not_bias,0))
              samples_true_bias = samples_true-samples_true_uniform
              bl = make_binary_list(total_samples, samples_true_uniform)
              bl = add_bias_one_set_values(bl,simulated_value_df, genename, percent_binary_attributes_thatarevaluebias, Zscore_valuebais, Zscore_valuebais_sigma, top_or_bottom, samples_true)
              bias_name = genename+"_Truesamplesize"+str(samples_true)+"_biassamplesize"+str(samples_true_bias)+"_Zscorevaluebais"+str(Zscore_valuebais)+"_sigma"+str(Zscore_valuebais_sigma)+"_direction_"+top_or_bottom+"_pba"+str(percent_binary_attributes_thatarevaluebias)
              ba_df = pd.DataFrame(index=simulated_value_df.index)
              ba_df[bias_name] = bl
              df_list.append(ba_df)
    sim_ba_df = pd.concat(df_list, axis=1)
    return ba_df




How many samples do you want to have the binary attribute? simulated_n_bianary_attributes = [20,40] means that the program will create some binary attributes that are true in 20 samples and some true in 40 samples.


In [21]:
#simulated_n_bianary_attributes = [20,40]
# I base my simulated_n_bianary_attributes on the real distrubution of the number of people with the comorbiditys
simulated_n_bianary_attributes = create_simulated_n_biarary_att_from_realdata(comorbid_df_metadata, nlevels_binaryatts=8)
simulated_n_bianary_attributes

[1, 11, 14, 19, 26, 27, 34, 141]

Do you want to simulate that the samples that are enriched for the biarary attribute have higher values (biasdirs=["bottom"]) or lower values (biasdirs=["top"]) or both (biasdirs=["top", "bottom"]) in the value data frame.

In [22]:
#biasdirs=["top"]
#biasdirs=["bottom"]
biasdirs=["top", "bottom"]

At what Zscore for the gene should the bias be at? What zscore-sigma should the bias have? Zscore_valuebais_list=[1,1.5,2,2.5] means the program will crease some biarary attributes that are bias at a zscore of 1 and some at a zcore of 1.5 relative to the mean and std of the value.

In [23]:
Zscore_valuebais_list=[1,1.5,2,2.5,3]
Zscore_valuebais_sigma_list=[0.5]


What percent of true samples do you want to be bais?

In [24]:
percent_binary_attributes_thatarevaluebias_list=range(0,100, 10)
percent_binary_attributes_thatarevaluebias_list=[i/100 for i in percent_binary_attributes_thatarevaluebias_list]
percent_binary_attributes_thatarevaluebias_list

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [27]:
simulated_ba_df = create_many_ba(simulated_value_df, percent_binary_attributes_thatarevaluebias_list, Zscore_valuebais_list, Zscore_valuebais_sigma_list, simulated_n_bianary_attributes, biasdirs)


In [28]:
simulated_ba_df

,simulated_based_on_ENSG00000279313_Truesamplesize141_biassamplesize0_Zscorevaluebais3_sigma0.5_direction_bottom_pba0.0
Adelaide,1
Alene,0
Alethea,0
Alex,0
Alfreda,0
...,...
Viviana,1
Willette,1
Wynne,0
Yettie,0


# Save output

In [29]:
simulated_ba_df.to_csv(outdir+"simulated_binary_attribute_"+date_time_string+".csv")

In [30]:
outdir

'/content/drive/MyDrive/temp/'